In [1]:
import pandas as pd
import folium

In [2]:
# CSV 파일 불러오기
csv_path = 'clean_data.csv'
df = pd.read_csv(csv_path)

# 데이터 확인
df.head()

,Company Name,Executive First Name,Executive Last Name,Address,City,State,Phone Number Combined,Primary SIC Description,SIC Code 1 Description,SIC Code 2 Description,Location Sales Volume Actual,Metro Area 1,Address1,lat,lon
0,Exotic Wings Chicken & Things,Chong,Kang,1254 Center Point Pkwy,Birmingham,AL,(205) 856-2900,Restaurants,Restaurants,NaN,"$345,000",Birmingham,1254 Center Point Pkwy,33.616676,-86.686055
1,Huntsville Hospital Physician,Heeeun,Kang,44 Hughes Rd # 1200,Madison,AL,(256) 265-5970,Physicians & Surgeons,Physicians & Surgeons,NaN,"$734,000",Huntsville,44 Hughes Rd,34.691088,-86.735355
2,Lotte Chemical,Richard,Kang,765 W Veterans Blvd,Auburn,AL,(334) 826-0094,Chemicals-Manufacturers,Chemicals-Manufacturers,NaN,"$632,000",Auburn-Oplka,765 W Veterans Blvd,32.561259,-85.509341
3,Mccorquodale Transfer Inc,Dong,Kang,2714 2nd Ave N,Birmingham,AL,(205) 969-0113,Storage-Household & Commercial,Storage-Household & Commercial,Delivery Service,"$10,820,000",Birmingham,2714 2nd Ave N,33.521350,-86.793737
4,One Solutions,Min,Kang,36 Samford Ave # D,Opelika,AL,(334) 737-6026,Personnel Consultants,Personnel Consultants,Employment Agencies & Opportunities,"$142,000",Auburn-Oplka,36 Samford Ave,32.657783,-85.372387


In [3]:
df.columns

Index(['Company Name', 'Executive First Name', 'Executive Last Name',
       'Address', 'City', 'State', 'Phone Number Combined',
       'Primary SIC Description', 'SIC Code 1 Description',
       'SIC Code 2 Description', 'Location Sales Volume Actual',
       'Metro Area 1', 'Address1', 'lat', 'lon'],
      dtype='object')

In [4]:
# 'lat' 또는 'lon' 중 하나라도 결측치를 가지고 있는 행들을 제거하여 new_df_filtered 생성
new_df_filtered = df.dropna(subset=['lat', 'lon']).copy()

# 결과 확인
new_df_filtered.head()

,Company Name,Executive First Name,Executive Last Name,Address,City,State,Phone Number Combined,Primary SIC Description,SIC Code 1 Description,SIC Code 2 Description,Location Sales Volume Actual,Metro Area 1,Address1,lat,lon
0,Exotic Wings Chicken & Things,Chong,Kang,1254 Center Point Pkwy,Birmingham,AL,(205) 856-2900,Restaurants,Restaurants,NaN,"$345,000",Birmingham,1254 Center Point Pkwy,33.616676,-86.686055
1,Huntsville Hospital Physician,Heeeun,Kang,44 Hughes Rd # 1200,Madison,AL,(256) 265-5970,Physicians & Surgeons,Physicians & Surgeons,NaN,"$734,000",Huntsville,44 Hughes Rd,34.691088,-86.735355
2,Lotte Chemical,Richard,Kang,765 W Veterans Blvd,Auburn,AL,(334) 826-0094,Chemicals-Manufacturers,Chemicals-Manufacturers,NaN,"$632,000",Auburn-Oplka,765 W Veterans Blvd,32.561259,-85.509341
3,Mccorquodale Transfer Inc,Dong,Kang,2714 2nd Ave N,Birmingham,AL,(205) 969-0113,Storage-Household & Commercial,Storage-Household & Commercial,Delivery Service,"$10,820,000",Birmingham,2714 2nd Ave N,33.521350,-86.793737
4,One Solutions,Min,Kang,36 Samford Ave # D,Opelika,AL,(334) 737-6026,Personnel Consultants,Personnel Consultants,Employment Agencies & Opportunities,"$142,000",Auburn-Oplka,36 Samford Ave,32.657783,-85.372387


In [7]:
# JSON 파일로 저장
new_df_filtered.to_json('data.json', orient='records')


In [8]:
# HTML 파일 생성
html_template = """
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8">
    <title>Interactive Map</title>
    <style>
        #map {
            height: 500px;
        }
    </style>
</head>
<body>
    <h2>Select a State</h2>
    <select id="stateSelect">
        <option value="">Select State</option>
        <option value="AL">AL</option>
        <option value="FL">FL</option>
        <option value="GA">GA</option>
        <option value="LA">LA</option>
        <option value="MS">MS</option>
        <option value="NC">NC</option>
        <option value="SC">SC</option>
        <option value="TN">TN</option>
        <option value="TX">TX</option>
    </select>

    <div id="map"></div>

    <script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.6.0/jquery.min.js"></script>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/leaflet/1.7.1/leaflet.js"></script>
    <script>
        $(document).ready(function() {
            // 지도 초기화
            var map = L.map('map').setView([37.09, -95.71], 4);

            // 지도 타일 레이어 추가
            L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
                maxZoom: 18,
                attribution: 'Map data &copy; <a href="https://www.openstreetmap.org/">OpenStreetMap</a> contributors'
            }).addTo(map);

            // State 선택 시 이벤트 핸들러
            $('#stateSelect').change(function() {
                var selectedState = $(this).val();
                showMarkers(selectedState);
            });

            // 초기 로딩 시에도 마커 표시
            var selectedState = $('#stateSelect').val();
            showMarkers(selectedState);

            // 선택된 State 값에 해당하는 마커만 지도에 표시하는 함수
            function showMarkers(state) {
                // 기존 마커 삭제
                map.eachLayer(function (layer) {
                    if (layer instanceof L.Marker) {
                        map.removeLayer(layer);
                    }
                });

                // 데이터 요청
                $.getJSON('data.json', function(data) {
                    // 선택된 State 값에 해당하는 마커 표시
                    data.forEach(function (location) {
                        if (location.state === state) {
                            L.marker([location.lat, location.lon]).addTo(map);
                        }
                    });
                });
            }
        });
    </script>
</body>
</html>
"""

# HTML 파일 저장
with open('map.html', 'w') as f:
    f.write(html_template)

UnicodeEncodeError: 'charmap' codec can't encode characters in position 968-969: character maps to <undefined>

In [9]:
import folium

# 지도 초기화
m = folium.Map()

# 데이터프레임을 순회하면서 마커 추가
for _, row in new_df_filtered.iterrows():
    lat = row['lat']
    lon = row['lon']
    company_name = row['Company Name']
    executive_first_name = row['Executive First Name']
    executive_last_name = row['Executive Last Name']
    address = row['Address']
    city = row['City']
    state = row['State']
    phone_number = row['Phone Number Combined']
    primary_sic_description = row['Primary SIC Description']
    sic_code_1_description = row['SIC Code 1 Description']
    sic_code_2_description = row['SIC Code 2 Description']
    location_sales_volume = row['Location Sales Volume Actual']
    metro_area = row['Metro Area 1']
    
    # 마커 생성
    marker = folium.Marker(
        location=[lat, lon],
        tooltip=f"{company_name}<br>{executive_first_name} {executive_last_name}<br>"
                f"{address}<br>{city}, {state}<br>{phone_number}<br>"
                f"{primary_sic_description}<br>{sic_code_1_description}<br>{sic_code_2_description}<br>"
                f"{location_sales_volume}<br>{metro_area}"
    )
    
    # 마커를 지도에 추가
    marker.add_to(m)

# 지도를 HTML 파일로 저장
m.save("ALL_MAP.html")
# 지도를 출력

